In [24]:
import requests
import pandas as pd
import plotly.express as px
from pprint import pprint


class WeatherAPI:
    weather_api_url = "https://api.weather.gov"

    def __init__(self, latitude, longitude, app_name, email):
        self.latitude = latitude
        self.longitude = longitude
        self.weather_api_headers = {"User-Agent": f"({app_name}, {email})"}

    def get_weather(self, path: str, use_full_url=False):
        r = requests.get(
            path if use_full_url else f"{self.weather_api_url}/{path}",
            headers=self.weather_api_headers,
        )

        return r.json()

    def get_local_weather(self):
        return self.get_weather(f"/points/{self.latitude},{self.longitude}")

    def get_local_hourly_forcast(self):
        local_weather = self.get_local_weather()
        hourly_forcast_url = local_weather["properties"]["forecastHourly"]
        hourly_forcast_data = self.get_weather(hourly_forcast_url, use_full_url=True)
        hourly_forcast_periods = hourly_forcast_data["properties"]["periods"]

        periods_useful_data = [
            {
                "startTime": period["startTime"],
                "temperature": period["temperature"],
                "relativeHumidity": period["relativeHumidity"]["value"],
                "probabilityOfPrecipitation": period["probabilityOfPrecipitation"][
                    "value"
                ],
                "windSpeed": int(str(period["windSpeed"]).split(" ")[0]),
            }
            for period in hourly_forcast_periods
        ]

        df = pd.DataFrame(periods_useful_data)
        df["startTime"] = pd.to_datetime(df["startTime"])
        
        return df


weather_client = WeatherAPI(
    latitude="41.8268",
    longitude="-71.5130",
    app_name="LocalWeatherApp",
    email="mastapegs01@gmail.com",
)

# Weather API

Source Documentation: https://www.weather.gov/documentation/services-web-api

## Authentication

A User Agent is required to identify your application. This string can be anything, and the more unique to your application the less likely it will be affected by a security event. If you include contact information (website or email), we can contact you if your string is associated to a security event. This will be replaced with an API key in the future.

`User-Agent: (myweatherapp.com, contact@myweatherapp.com)`

In [25]:
weather_data = weather_client.get_weather("")


pprint(weather_data)

{'status': 'OK'}


## Local Coordinates

02919/Coordinates
41.8268° N, 71.5130° W

`https://api.weather.gov/points/{latitude},{longitude}`
For example: https://api.weather.gov/points/39.7456,-97.0892

In [26]:
df = weather_client.get_local_hourly_forcast()
df

,startTime,temperature,relativeHumidity,probabilityOfPrecipitation,windSpeed
0,2024-04-29 11:00:00-04:00,71,61,1,9
1,2024-04-29 12:00:00-04:00,72,57,1,9
2,2024-04-29 13:00:00-04:00,74,51,2,8
3,2024-04-29 14:00:00-04:00,75,48,2,8
4,2024-04-29 15:00:00-04:00,73,49,4,7
...,...,...,...,...,...
151,2024-05-05 18:00:00-04:00,58,69,24,12
152,2024-05-05 19:00:00-04:00,57,72,25,12
153,2024-05-05 20:00:00-04:00,55,74,25,12
154,2024-05-05 21:00:00-04:00,54,77,25,12


## Plotting the useful data

In [27]:
fig = px.scatter(
    df,
    x="startTime",
    y="temperature",
    color="probabilityOfPrecipitation",
    size="windSpeed",
    title="Johnston/Smithfield Hourly Weather Forcast",
    trendline="lowess",
    trendline_color_override="black",
    trendline_options=dict(frac=0.15),
)
fig.show()